# ch.paf.link Lite

Goal is to try a very lightweight ch.paf.link schema that would present a lower adoption barrier.

## Design Principles

- Activities group together what happened at one time (prov:atTime) or in one time interval (prov:startedAtTime and prov:endedAtTime) and was executed by agents described by a role that in itself clearly defines their contribution to the activity --> time and agend is mandatory
- Activities use entities as input and produce entities as output. A single entity can be produced and used at the same time by an activity. It is not per se defined, what input and output means but has to be defined in the application

In [1]:
from rdf_notebook import query_ttl

## Office Consultation

### Most Basic Model

- Entities are not created, but only used
- Usage of entities is not repeated within activity stream
- No very specific chpaf classes, use paf classes with an additional general chpaf:OfficeConsultation class

In [10]:
ttl = """

@prefix chpaf: <https://ch.paf.link/> .
@prefix paf: <https://paf.link/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix schema: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# identifier entity - no creation
<https://politics.ld.admin.ch/parliamentary-affair/23.4022> a paf:IdentifierEntity, chpaf:ParliamentaryAffair ;
    schema:identifier "23.4022" .

# entity representing the consultation itself - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/entity> a paf:ConsultationEntity, chpaf:OfficeConsultation ;
    schema:description "Description of the consultation" ;
    schema:name "Title of the consultation" .

# activity for sending the consultation - with start and end time
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> a paf:ConsultationActivity, chpaf:OfficeConsultation ;
    prov:startetAtTime "2023-12-18"^^xsd:date ;
    prov:endedAtTime "2024-01-15"^^xsd:date ;
    prov:used
        <https://politics.ld.admin.ch/parliamentary-affair/23.4022> ,
        <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/entity> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/III.1.4> ;
        prov:hadRole chpaf:OfficeConsultationSender ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.2> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.3> ;
        prov:hadRole chpaf:OfficeConsultationReceiver ;
    ] .

# entity representing a comment - no creation
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1> a paf:CommentEntity, chpaf:OfficeConsultation ;
    schema:text "This is a comment..." .

# activity for commenting
<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-1> a paf:CommentActivity, chpaf:OfficeConsultation ;
    prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
    prov:atTime "2024-01-10T10:00:00"^^xsd:dateTime ;
    prov:used <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1> ;
    prov:qualifiedAssociation [
        a prov:Association ;
        prov:agent <https://ld.admin.ch/office/II.1.2> ;
        prov:hadRole chpaf:OfficeConsultationCommentator ;
    ] .

"""

### Querying

- All comments to a certain consultation

In [3]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    ?comment_entity a paf:CommentEntity, chpaf:OfficeConsultation ;
        ^prov:used/prov:wasInformedBy <https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> ;
        schema:text ?comment_text .
}

"""

query_ttl(ttl, query)

,comment_entity,comment_text
0,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-entity-1,This is a comment...


- Receivers that did comment

In [11]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT ?agent WHERE {

    BIND(<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> AS ?consultation_activity) .
    
    ?consultation_activity prov:qualifiedAssociation ?receiver_assoc .
    ?receiver_assoc prov:agent ?agent ;
        prov:hadRole chpaf:OfficeConsultationReceiver .

    ?comment_activity prov:wasInformedBy ?consultation_activity ; 
        prov:qualifiedAssociation ?commentator_assoc .
    ?commentator_assoc prov:agent ?agent ;
        prov:hadRole chpaf:OfficeConsultationCommentator .
}

"""

query_ttl(ttl, query)

,agent
0,https://ld.admin.ch/office/II.1.2


- Receivers that did not comment

In [12]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT ?agent WHERE {

    BIND(<https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity> AS ?consultation_activity) .
    
    ?consultation_activity prov:qualifiedAssociation ?receiver_assoc .
    ?receiver_assoc prov:agent ?agent ;
        prov:hadRole chpaf:OfficeConsultationReceiver .
    
    MINUS {    
    
        ?comment_activity prov:wasInformedBy ?consultation_activity ; 
            prov:qualifiedAssociation ?commentator_assoc .
        ?commentator_assoc prov:agent ?agent ;
            prov:hadRole chpaf:OfficeConsultationCommentator .
    }

}

"""

query_ttl(ttl, query)

,agent
0,https://ld.admin.ch/office/II.1.3


- Check whether comment is within the correct date

In [27]:
query = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>

SELECT * WHERE {

    ?comment_activity a paf:CommentActivity, chpaf:OfficeConsultation ;
        prov:atTime ?comment_time ;
        prov:wasInformedBy ?consultation_activity .

    ?consultation_activity a paf:ConsultationActivity, chpaf:OfficeConsultation ;
        prov:startetAtTime ?start_date ;
        prov:endedAtTime ?end_date .

    # casting to xsd:dateTime for comparison works only after casting to string first in rdflib
    BIND(IF(?comment_time >= xsd:dateTime(STR(?start_date)) && ?comment_time <= xsd:dateTime(STR(?end_date)), "within timeframe", "out of timeframe") AS ?status)
}

"""

query_ttl(ttl, query)

,status,end_date,start_date,consultation_activity,comment_activity,comment_time
0,within timeframe,2024-01-15,2023-12-18,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/consultation-activity,https://politics.ld.admin.ch/office-consultation/OCo_2023.2157/comment-activity-1,2024-01-10T10:00:00
